In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from dotenv import load_dotenv
import os
import sys
import logging

In [2]:
spark = SparkSession.builder \
    .appName("TaxiTrips") \
    .getOrCreate()

25/04/25 11:10:38 WARN Utils: Your hostname, brempong-HP-EliteBook-840-G7-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.36.94 instead (on interface wlp0s20f3)
25/04/25 11:10:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 11:10:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
trip_start = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../data/trip_start.csv")

In [5]:
trip_start.show()

+----------+------------------+-------------------+---------+-------------------+--------------------------+---------------------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|    pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|
+----------+------------------+-------------------+---------+-------------------+--------------------------+---------------------+
|c66ce556bc|                93|                 93|        1|2024-05-25 13:19:00|       2024-05-25 14:03:00|    34.18595283806629|
|f93c901b2d|               247|                247|        1|2024-05-25 18:32:00|       2024-05-25 19:15:00|    69.12471722003522|
|9ddba1d91a|                 7|                193|        2|2024-05-25 05:17:00|       2024-05-25 05:37:00|    47.09790128664532|
|cc4db3afb9|               193|                193|        2|2024-05-25 16:16:00|       2024-05-25 17:02:00|    66.54630432611239|
|f881b97b2c|                93|                 93|        1|2024-05-25 12:43:00|  

In [6]:
trip_start = trip_start.withColumn(
    "pickup_time",
    date_format("pickup_datetime", "HH:mm:ss")
)
trip_start.show()

+----------+------------------+-------------------+---------+-------------------+--------------------------+---------------------+-----------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|    pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|pickup_time|
+----------+------------------+-------------------+---------+-------------------+--------------------------+---------------------+-----------+
|c66ce556bc|                93|                 93|        1|2024-05-25 13:19:00|       2024-05-25 14:03:00|    34.18595283806629|   13:19:00|
|f93c901b2d|               247|                247|        1|2024-05-25 18:32:00|       2024-05-25 19:15:00|    69.12471722003522|   18:32:00|
|9ddba1d91a|                 7|                193|        2|2024-05-25 05:17:00|       2024-05-25 05:37:00|    47.09790128664532|   05:17:00|
|cc4db3afb9|               193|                193|        2|2024-05-25 16:16:00|       2024-05-25 17:02:00|    66.54630432611239|   16:16:00|

In [7]:
trip_start = trip_start.withColumn("pickup_datetime", col("pickup_datetime").cast("date"))
trip_start.show()


+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|pickup_time|
+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+
|c66ce556bc|                93|                 93|        1|     2024-05-25|       2024-05-25 14:03:00|    34.18595283806629|   13:19:00|
|f93c901b2d|               247|                247|        1|     2024-05-25|       2024-05-25 19:15:00|    69.12471722003522|   18:32:00|
|9ddba1d91a|                 7|                193|        2|     2024-05-25|       2024-05-25 05:37:00|    47.09790128664532|   05:17:00|
|cc4db3afb9|               193|                193|        2|     2024-05-25|       2024-05-25 17:02:00|    66.54630432611239|   16:16:00|
|f881b97b2c|               

In [8]:
trip_start = trip_start.withColumn("estimated_fare_amount", round(col("estimated_fare_amount"), 2))
trip_start.show()

+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|pickup_time|
+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+
|c66ce556bc|                93|                 93|        1|     2024-05-25|       2024-05-25 14:03:00|                34.19|   13:19:00|
|f93c901b2d|               247|                247|        1|     2024-05-25|       2024-05-25 19:15:00|                69.12|   18:32:00|
|9ddba1d91a|                 7|                193|        2|     2024-05-25|       2024-05-25 05:37:00|                 47.1|   05:17:00|
|cc4db3afb9|               193|                193|        2|     2024-05-25|       2024-05-25 17:02:00|                66.55|   16:16:00|
|f881b97b2c|               

In [9]:
trip_end = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../data/trip_end.csv")
trip_end.show()

+-------------------+---------+---------------+-------------+------------------+------------------+------------+---------+----------+
|   dropoff_datetime|rate_code|passenger_count|trip_distance|       fare_amount|        tip_amount|payment_type|trip_type|   trip_id|
+-------------------+---------+---------------+-------------+------------------+------------------+------------+---------+----------+
|2024-05-25 14:05:00|      5.0|            0.0|          0.1| 40.09620692463166|               0.0|         3.0|      2.0|c66ce556bc|
|2024-05-25 19:15:00|      5.0|            1.0|          0.0| 69.12471722003522|               0.0|         3.0|      2.0|f93c901b2d|
|2024-05-25 05:41:00|      1.0|            1.0|          0.0| 47.09790128664532|               0.0|         1.0|      1.0|9ddba1d91a|
|2024-05-25 17:05:00|      1.0|            1.0|          0.0|   77.156183662551|1.4231808570380187|         1.0|      1.0|cc4db3afb9|
|2024-05-25 13:39:00|      5.0|            1.0|          0.1| 

In [10]:
trip_end.select(col("dropoff_datetime")).distinct().show()

+-------------------+
|   dropoff_datetime|
+-------------------+
|2024-05-25 16:05:00|
|2024-05-25 09:54:00|
|2024-05-25 09:46:00|
|2024-05-25 03:12:00|
|2024-05-25 06:12:00|
|2024-05-25 08:12:00|
|2024-05-25 17:13:00|
|2024-05-26 00:24:00|
|2024-05-25 20:20:00|
|2024-05-25 02:23:00|
|2024-05-25 23:48:00|
|2024-05-26 00:25:00|
|2024-05-25 23:24:00|
|2024-05-25 23:05:00|
|2024-05-25 11:54:00|
|2024-05-25 01:49:00|
|2024-05-25 06:22:00|
|2024-05-26 00:14:00|
|2024-05-25 18:34:00|
|2024-05-25 15:20:00|
+-------------------+
only showing top 20 rows



In [11]:
trip_end = trip_end.withColumn(
    "dropoff_time",
    date_format("dropoff_datetime", "HH:mm:ss")
)
trip_end.show()

+-------------------+---------+---------------+-------------+------------------+------------------+------------+---------+----------+------------+
|   dropoff_datetime|rate_code|passenger_count|trip_distance|       fare_amount|        tip_amount|payment_type|trip_type|   trip_id|dropoff_time|
+-------------------+---------+---------------+-------------+------------------+------------------+------------+---------+----------+------------+
|2024-05-25 14:05:00|      5.0|            0.0|          0.1| 40.09620692463166|               0.0|         3.0|      2.0|c66ce556bc|    14:05:00|
|2024-05-25 19:15:00|      5.0|            1.0|          0.0| 69.12471722003522|               0.0|         3.0|      2.0|f93c901b2d|    19:15:00|
|2024-05-25 05:41:00|      1.0|            1.0|          0.0| 47.09790128664532|               0.0|         1.0|      1.0|9ddba1d91a|    05:41:00|
|2024-05-25 17:05:00|      1.0|            1.0|          0.0|   77.156183662551|1.4231808570380187|         1.0|      

In [12]:
trip_end = trip_end.withColumn("dropoff_datetime", col("dropoff_datetime").cast("date"))
trip_end.show()


+----------------+---------+---------------+-------------+------------------+------------------+------------+---------+----------+------------+
|dropoff_datetime|rate_code|passenger_count|trip_distance|       fare_amount|        tip_amount|payment_type|trip_type|   trip_id|dropoff_time|
+----------------+---------+---------------+-------------+------------------+------------------+------------+---------+----------+------------+
|      2024-05-25|      5.0|            0.0|          0.1| 40.09620692463166|               0.0|         3.0|      2.0|c66ce556bc|    14:05:00|
|      2024-05-25|      5.0|            1.0|          0.0| 69.12471722003522|               0.0|         3.0|      2.0|f93c901b2d|    19:15:00|
|      2024-05-25|      1.0|            1.0|          0.0| 47.09790128664532|               0.0|         1.0|      1.0|9ddba1d91a|    05:41:00|
|      2024-05-25|      1.0|            1.0|          0.0|   77.156183662551|1.4231808570380187|         1.0|      1.0|cc4db3afb9|    17

In [13]:
trip_end = trip_end.withColumn("fare_amount", round(col("fare_amount"), 2))
trip_end.show()

+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+----------+------------+
|dropoff_datetime|rate_code|passenger_count|trip_distance|fare_amount|        tip_amount|payment_type|trip_type|   trip_id|dropoff_time|
+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+----------+------------+
|      2024-05-25|      5.0|            0.0|          0.1|       40.1|               0.0|         3.0|      2.0|c66ce556bc|    14:05:00|
|      2024-05-25|      5.0|            1.0|          0.0|      69.12|               0.0|         3.0|      2.0|f93c901b2d|    19:15:00|
|      2024-05-25|      1.0|            1.0|          0.0|       47.1|               0.0|         1.0|      1.0|9ddba1d91a|    05:41:00|
|      2024-05-25|      1.0|            1.0|          0.0|      77.16|1.4231808570380187|         1.0|      1.0|cc4db3afb9|    17:05:00|
|      2024-05-25|      5.0|            1

In [14]:
trips = trip_start.join(trip_end, on="trip_id", how="inner")
trips.show()

+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+------------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|pickup_time|dropoff_datetime|rate_code|passenger_count|trip_distance|fare_amount|        tip_amount|payment_type|trip_type|dropoff_time|
+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+------------+
|c66ce556bc|                93|                 93|        1|     2024-05-25|       2024-05-25 14:03:00|                34.19|   13:19:00|      2024-05-25|      5.0|            0.0|          0.1|       40.1|    

In [15]:
trips.select("dropoff_datetime").distinct().show(truncate=False)


+----------------+
|dropoff_datetime|
+----------------+
|2024-05-25      |
|2024-05-26      |
+----------------+



In [16]:
from pyspark.sql.functions import col

completed_trips = trips.filter(
     col("dropoff_datetime").isNotNull()
)
completed_trips.show()

+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+------------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|pickup_time|dropoff_datetime|rate_code|passenger_count|trip_distance|fare_amount|        tip_amount|payment_type|trip_type|dropoff_time|
+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+------------+
|c66ce556bc|                93|                 93|        1|     2024-05-25|       2024-05-25 14:03:00|                34.19|   13:19:00|      2024-05-25|      5.0|            0.0|          0.1|       40.1|    

In [17]:
daily_kpis = trips.groupBy("pickup_datetime").agg(
    sum("fare_amount").alias("total_fare"),
    count("fare_amount").alias("count_trips"),
    avg("fare_amount").alias("average_fare"),
    max("fare_amount").alias("max_fare"),
    min("fare_amount").alias("min_fare")
)

daily_kpis.show()

+---------------+-----------------+-----------+-----------------+--------+--------+
|pickup_datetime|       total_fare|count_trips|     average_fare|max_fare|min_fare|
+---------------+-----------------+-----------+-----------------+--------+--------+
|     2024-05-25|274393.9300000005|       4999|54.88976395279066|   99.99|   10.04|
+---------------+-----------------+-----------+-----------------+--------+--------+



In [18]:
"""" completed_trips = completed_trips.filter(
    col("rate_code").isNotNull() & 
    col("passenger_count").isNotNull() & 
    col("payment_type").isNotNull() & 
    col("trip_type").isNotNull()
)
completed_trips.show()"""

'" completed_trips = completed_trips.filter(\n    col("rate_code").isNotNull() & \n    col("passenger_count").isNotNull() & \n    col("payment_type").isNotNull() & \n    col("trip_type").isNotNull()\n)\ncompleted_trips.show()'

In [19]:
completed_trips = completed_trips.repartition(2, col("dropoff_datetime"))#.sortWithinPartitions(
    #col("rate_code"))

completed_trips.show()

+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+------------+
|   trip_id|pickup_location_id|dropoff_location_id|vendor_id|pickup_datetime|estimated_dropoff_datetime|estimated_fare_amount|pickup_time|dropoff_datetime|rate_code|passenger_count|trip_distance|fare_amount|        tip_amount|payment_type|trip_type|dropoff_time|
+----------+------------------+-------------------+---------+---------------+--------------------------+---------------------+-----------+----------------+---------+---------------+-------------+-----------+------------------+------------+---------+------------+
|c66ce556bc|                93|                 93|        1|     2024-05-25|       2024-05-25 14:03:00|                34.19|   13:19:00|      2024-05-25|      5.0|            0.0|          0.1|       40.1|    

In [20]:
daily_kpis = completed_trips.groupBy("pickup_datetime").agg(
    sum("fare_amount").alias("total_fare"),
    count("fare_amount").alias("count_trips"),
    avg("fare_amount").alias("average_fare"),
    max("fare_amount").alias("max_fare"),
    min("fare_amount").alias("min_fare")
)

daily_kpis.show()


+---------------+-----------------+-----------+-----------------+--------+--------+
|pickup_datetime|       total_fare|count_trips|     average_fare|max_fare|min_fare|
+---------------+-----------------+-----------+-----------------+--------+--------+
|     2024-05-25|274393.9300000005|       4999|54.88976395279066|   99.99|   10.04|
+---------------+-----------------+-----------+-----------------+--------+--------+

